# Similarity Score

In [1]:
from zalign import ZScore
from rdkit import Chem


fp_sdf_q = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/CAT-13g_rand.sdf"
fp_sdf_t = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/CAT-24.sdf"

mol_q = Chem.SDMolSupplier(fp_sdf_q, removeHs=False)[0]
mol_t = Chem.SDMolSupplier(fp_sdf_t, removeHs=False)[0]

In [2]:
mol_t.GetConformer().GetPositions().tolist()[0]

[15.3153, 5.5063, 2.895]

In [ ]:
matrix = ZScore(mol_q, mol_t).score()

[15.3153, 5.5063, 2.895]
[15.3153, 5.5063, 2.895]
[15.3153, 5.5063, 2.895]
[15.3153, 5.5063, 2.895]


In [4]:
mol_t.GetConformer().GetPositions().tolist()[0]

[15.3153, 5.5063, 2.895]

In [87]:
# 用plotly绘制相似性得分矩阵的热力图，可以交互式查看横纵坐标
import plotly.graph_objs as go
import numpy as np

# 构造x轴和y轴标签（原子索引从1开始）
x_labels = [str(i+1) for i in range(matrix.shape[1])]
y_labels = [str(i+1) for i in range(matrix.shape[0])]

heatmap = go.Heatmap(
    z=matrix,
    x=x_labels,
    y=y_labels,
    colorscale='Hot',
    colorbar=dict(title='Score'),
    hoverongaps=False,
    hovertemplate='Query Atom: %{y}<br>Template Atom: %{x}<br>Score: %{z:.1f}<extra></extra>'
)

layout = go.Layout(
    title='Similarity Score Matrix',
    xaxis=dict(title='Template Molecule Atom Index'),
    yaxis=dict(title='Query Molecule Atom Index'),
    width=900,
    height=900
)

fig = go.Figure(data=[heatmap], layout=layout)
fig.show()

## 准备输入JSON

In [ ]:
import json
import os

dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/"
fp = os.path.join(dp, "tmp", os.listdir(os.path.join(dp, "tmp"))[0], "ud2_engine_inputs.json")
fp_align = os.path.join(dp, "ud2_align.json")

with open(fp, "r") as f:
    data = json.load(f)

k = list(set(data.keys()) - {"score", "receptor"})[0]

# 得到所有原子的坐标
coords_t = mol_t.GetConformer().GetPositions().tolist()

In [95]:
mol_t.GetConformer().GetPositions().tolist()

[[6.958391496922472, 0.5501643032665513, -2.91028824161002],
 [6.072768032503382, 0.02244863910145944, -1.8716111176763575],
 [5.35663895298835, -0.41283316239171486, -1.011121006780161],
 [4.512612067613621, -0.9310891033363213, 0.018015595851899137],
 [4.849222058758155, -2.0881217133540644, 0.7051206705979235],
 [4.089973156505257, -2.625737692519205, 1.688159467727291],
 [2.9506187544638913, -1.966790713334117, 2.004352187640291],
 [2.508415844996216, -0.8008969245240866, 1.3764349224846595],
 [1.2463184345198124, -0.1560340091047169, 1.7527073538749294],
 [1.0520425561486457, 0.3460096557192944, 3.044747502441451],
 [-0.1501417002806911, 0.9561361666587422, 3.3959291446633633],
 [-1.180232371809518, 1.065719514466302, 2.462035723074447],
 [-1.032447862796958, 0.5633065370800652, 1.1570281185215616],
 [-2.1832564295693664, 0.6901369316380308, 0.16866854310046792],
 [-2.39120354087319, 2.126979710677108, -0.2405589320826936],
 [-3.6064328139970834, 2.5658906481348227, -1.02779852344

In [91]:
natom_t = mol_t.GetNumAtoms()
natom_q = mol_q.GetNumAtoms()

for i in range(natom_q):

    biases = []
    for j in range(natom_t): # each template atom
        biases.append([*coords_t[j], float(matrix[i][j]), 0.])
    
    # add biases to this
    data[k]["atoms"][i].append(biases)

with open(fp_align, "w") as f:
    json.dump(data, f)

In [92]:
print(data[k]["atoms"][31][-1])

[[6.296927228490213, 2.035312442962001, -3.1301992550357274, 2.84375, 0.0], [5.649002039669799, 1.238912525013319, -2.087331886385702, 2.0, 0.0], [5.131309701808326, 0.5757014352799695, -1.2299931500505163, 2.0, 0.0], [4.526416529990437, -0.2201477246211226, -0.2096493343045087, 0.6694582499999999, 0.0], [5.259291903780828, -0.6623469636212118, 0.8819934248573869, 0.9927499999999999, 0.0], [4.741318435023449, -1.4184085317272672, 1.8777549376687868, 0.0, 0.0], [3.4347724497669008, -1.7549894556225973, 1.7685396988465432, 0.9927499999999999, 0.0], [2.6007385273468913, -1.3621096342500214, 0.7203606010980061, 0.15625, 0.0], [1.186287876759058, -1.7445991743877811, 0.6790320165913276, 0.15625, 0.0], [0.8083342701934912, -3.087975728629886, 0.5717747513051893, 0.0, 0.0], [-0.5375328510535626, -3.445840194775626, 0.5375996077985855, 0.0, 0.0], [-1.5260256002179466, -2.465105400057951, 0.6162388725610558, 0.0, 0.0], [-1.1903718889412305, -1.1041733115736632, 0.7302457623755937, 0.0, 0.0], [-

# Utils

## Transfer .MOL file to .SDF File

In [22]:
def transfer_mol_to_sdf(mol_file_path):
    fp_sdf = mol_file_path.replace(".mol", ".sdf")
    mol = Chem.MolFromMolFile(mol_file_path)
    Chem.MolToMolFile(mol, fp_sdf)
    return fp_sdf

transfer_mol_to_sdf("/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/test_all_rings.mol")

'/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/test_all_rings.sdf'

# Attractive Energy

# Repulsive Energy